In [1]:
is_it_a_test = True
test_name = "_feature"

In [2]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

import os

print(os.getcwd())


def update_working_directory():
    from pathlib import Path

    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)


update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


<IPython.core.display.Javascript object>

In [3]:
import numpy as np
import pandas as pd

from src.data.collect_data import *

<IPython.core.display.Javascript object>

In [4]:
vocab = initiate_vocab(test=is_it_a_test, test_name=test_name)

historical_data, tries_session, nb_known_words_session = initiate_session()

<IPython.core.display.Javascript object>

In [5]:
for i in range(100):

    # Choose language
    input_language, output_language = choose_a_language(vocab)
    if input_language == -1:
        break

    # Pick a random word
    id_vocab = choose_a_word(vocab, input_language, output_language)

    # Take a guess
    your_guess, question_time = prompt_question(
        vocab,
        id_vocab,
        input_language,
        output_language,
        nb_known_words_session,
        tries_session,
    )

    # Answer analysis
    is_it_correct = (
        your_guess
        == vocab.loc[vocab["id_vocab"] == id_vocab, output_language].values[0]
    )
    is_it_another_word = your_guess in vocab[output_language].values

    # Do we want to stop?
    if your_guess == "STOP IT":
        print(
            "Your last word was '{}={}'".format(
                vocab.loc[a, input_language].values[0],
                vocab.loc[vocab["id_vocab"] == id_vocab, output_language].values[0],
            )
        )
        break

    # If correct answer
    if is_it_correct:
        if_correct()
        nb_known_words_session += 1
        write_it_again = None

    # If wrong answer
    if not is_it_correct:
        write_it_again = if_not_correct(
            vocab,
            id_vocab,
            input_language,
            output_language,
            your_guess,
            is_it_another_word,
        )

    # Add to historical data
    historical_data = add_historical_data(
        historical_data,
        vocab,
        id_vocab,
        input_language,
        output_language,
        is_it_correct,
        your_guess,
        question_time,
        write_it_again,
    )

    # Update vocab
    vocab = update_vocab(
        vocab, id_vocab, input_language, output_language, is_it_correct
    )

    tries_session += 1

0/0 - What is the english word for 'rennen'?   to sprint
Correct!
1/1 - What is the german word for 'welcome'?   wilkommen
Correct!
2/2 - What is the english word for 'verheiratet'?   marrid
Sorry, it was 'verheiratet = married'
Write it again   
2/3 - What is the english word for 'September'?   September
Correct!
3/4 - What is the german word for 'a lot'?   viel
Correct!
4/5 - What is the german word for 'the year'?   die Jahr
Sorry, it was 'the year = das Jahr'
Write it again   
4/6 - What is the english word for 'zusammen'?   together
Correct!
5/7 - What is the german word for 'to speak'?   sprechen
Correct!
6/8 - What is the english word for 'wirklich'?   wirkclih
Sorry, it was 'wirklich = really'
Write it again   
6/9 - What is the english word for 'die Aprikose'?   the apricot
Correct!
7/10 - What is the english word for 'die Ärztin'?   the femal ocrg
Sorry, it was 'die Ärztin = the female doctor'
Write it again   
7/11 - What is the english word for 'richtig'?   correct
Correct!

<IPython.core.display.Javascript object>

In [6]:
# Update and save historical data
update_historical_data(historical_data, test=is_it_a_test, test_name=test_name)

<IPython.core.display.Javascript object>

In [7]:
# Update and save vocab
finalize_vocab(vocab, test=is_it_a_test, test_name=test_name)

<IPython.core.display.Javascript object>